In [40]:
"""
Needs:

1. Produce a file containing the average pages 
per minute and average pages per session 
for each IP

Each session ends after 20 min of inactivity

"""

import csv
from datetime import datetime, timedelta
from operator import itemgetter

with open('distilAccessLog.tsv','r') as tsv:
    AccessLogList = [line.strip().split('\t') for line in tsv]

In [41]:
for item in AccessLogList:
    item[1] = datetime.utcfromtimestamp(float(item[1]))

# Sort the list by timestamp
sorted(AccessLogList, key=itemgetter(1))

[['24.234.66.218',
  datetime.datetime(2016, 1, 13, 22, 56, 38, 463000),
  'GET /page.143678790852849234.html',
  '200',
  'Mozilla/5.0 (Linux; Android 5.1.1; SM-T237P Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Safari/537.36'],
 ['51.39.65.224',
  datetime.datetime(2016, 1, 13, 22, 56, 38, 551000),
  'GET /page.71615.9647734462844.html',
  '200',
  'Mozilla/5.0 (iPhone; CPU iPhone OS 9_2 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) GSA/5.4.49956 Mobile/13C75 Safari/600.1.4'],
 ['199.21.254.3',
  datetime.datetime(2016, 1, 13, 22, 56, 38, 553000),
  'GET /page.27050.9644270242335.html',
  '404',
  'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)'],
 ['40.77.167.19',
  datetime.datetime(2016, 1, 13, 22, 56, 38, 599000),
  'GET /page.921836295960196604.html',
  '200',
  'Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)'],
 ['157.55.39.114',
  datetime.datetime(2016, 1, 13, 22, 56, 38, 605000),
  'GET /page.320

In [50]:
session_dict = {}

for item in AccessLogList:
    # If we haven't add to the session dict, store session_id as 1, timespent, pages first session, last session, pages, last page
    if item[0] not in session_dict.keys():
        session_dict[item[0]] = [[1, 0, item[1], item[1], 1, item[2]]]
    else:
        # If within 20 minutes of last session, then update the length of time spent
        if item[1] < session_dict[item[0]][-1][3] + timedelta(minutes=20):
            # Update the total timespent
            session_dict[item[0]][-1][1] = (item[1] - session_dict[item[0]][-1][2]).total_seconds() / 60.0
            
            # Update the last timestamp
            session_dict[item[0]][-1][3] = item[1]
            
            # Update the page
            if item[2] != session_dict[item[0]][-1][5]:
                session_dict[item[0]][-1][5] = item[2]
                session_dict[item[0]][-1][4] += 1
        
        # Count as new session
        else:
            # Increment the session id from the last one and update the fields
            session_dict[item[0]].append([session_dict[item[0]][-1][0] + 1, 0,item[1], item[1], 1, item[2]])

In [56]:
"""
Need:

Avg pages per minute
Avg pages per session
"""

dim_ip_activities = {}
for key in session_dict.keys():
    if key not in dim_ip_activities.keys():
        pages = 0
        sessions = session_dict[key][-1][0]
        minutes = 0
        for item in session_dict[key]:
            pages += item[4]
            minutes += item[1]      
        
        # Default to one minute if the aggregate number of minutes is 0
        minutes = minutes if minutes != 0 else 1
        dim_ip_activities[key] = [float(pages)/minutes, float(pages)/sessions]

{'24.234.66.218': [1.0, 1.0], '51.39.65.224': [86.71912638509716, 9.0], '199.21.254.3': [1.2892689844857967, 2.0], '40.77.167.19': [97.91496675404942, 282.0], '157.55.39.114': [58.53279415157808, 241.0], '50.140.172.248': [8.919280511372081, 6.0], '66.86.129.102': [249.1349480968858, 6.0], '107.77.106.56': [1.0, 1.0], '66.249.79.119': [84.6494336645315, 422.0], '104.251.247.231': [1.0, 1.0], '208.54.90.244': [157.1966595709841, 16.0], '108.17.26.19': [0.44435721876816775, 2.0], '184.97.187.61': [258.2496413199426, 21.0], '54.208.61.227': [0.83965112495758, 4.0], '209.234.190.250': [0.444413170925009, 2.0], '75.44.64.9': [0.44446090595948, 2.0], '66.182.213.194': [7.993095818684152, 23.0], '98.163.212.215': [0.44458440620195244, 2.0], '98.225.98.137': [188.08777429467085, 2.0], '108.129.177.2': [68.05614632071459, 8.0], '62.210.141.46': [1.0, 1.0], '64.161.153.34': [8.388881331338139, 37.0], '208.54.35.151': [1.0, 1.0], '76.77.213.3': [6.640035127927773, 31.0], '5.9.17.118': [1.26951885

In [63]:
"""
Output to a CSV file
"""
dim_ip_activities_list = [[key] + dim_ip_activities[key] for key in dim_ip_activities.keys()]

with open("dim_ip_activities.csv",'w') as resultFile:
    wr = csv.writer(resultFile, dialect='excel')
    wr.writerows(dim_ip_activities_list)